In [1]:
pip install transformers tensorflow sentencepiece category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from numba import cuda
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate
import copy
import category_encoders as ce
from numpy import unique
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import concatenate
from keras.utils import plot_model
import math
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM
from transformers import XLMRobertaTokenizer, TFAutoModel
from transformers import TFAutoModelForSequenceClassification
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
#from keras.utils import plot_model
import os
import torch
from huggingface_hub import from_pretrained_keras
from collections import Counter
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import losses
from transformers import pipeline
from keras.layers import Softmax
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras import regularizers
from keras import backend as K
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

In [3]:
#pandas settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
#load the data
df = pd.read_pickle("/content/drive/MyDrive/Thesis/1.data/4C.df_structured_unstructured").reset_index(drop = True)

In [6]:
def encode(df):
    #copy date column for sorting
    df["date copy"] = df["DT_AWARD"].copy(deep=True)

    #ENCODE COLUMNS
    base_n_encoder_cols = ["DT_AWARD", "TAL_LOCATION_NUTS"]
    one_hot_cols = ["TYPE_OF_CONTRACT", "TOP_TYPE", "B_ACCELERATED", "ISO_COUNTRY_CODE", "CAE_TYPE", "MAIN_ACTIVITY", "B_ON_BEHALF", "B_AWARDED_BY_CENTRAL_BODY", "B_FRA_AGREEMENT", "B_EU_FUNDS", "CRIT_CODE", "CPV"]
    df = pd.get_dummies(df, columns=one_hot_cols, drop_first=True, dtype=int)
    encoder = ce.BaseNEncoder(cols=base_n_encoder_cols, return_df=True, base=2)
    df = encoder.fit_transform(df)

    return df

In [7]:
def scale(df):
    numeric_cols = ["NUMBER_OFFERS", "AWARD_EST_VALUE_EURO", "CRIT_PRICE_WEIGHT"]
    scaler = RobustScaler()

    values = df[numeric_cols].values.astype(float)
    scaled_values = scaler.fit_transform(values)
    df.loc[:,numeric_cols] = np.array(scaled_values)

    return df

In [8]:
def encode_scale_split(df, train, validate, text_column):
    """encode scale and split dataset"""
    #ENCODE
    df = encode(df)
    df = df.sort_values(by = ["date copy"], axis = 0, ascending = True)
    train_indice = int(train * len(df))
    validate_indice = train_indice + int(validate * len(df))
    df = df.drop(columns = ["date copy"])

    train_set = df.iloc[:train_indice]
    val_set = df.iloc[train_indice:validate_indice]
    test_set = df.iloc[validate_indice:]

    #PERFORM SCALING
    train_set = scale(train_set)
    val_set = scale(val_set)
    test_set = scale(test_set)

    #SPLIT THE DATASET
    target_col = "AWARD_VALUE_EURO_FIN_1"
    X_train = train_set.drop(columns = [target_col]).values
    X_train_text = train_set[[text_column]].values.flatten().tolist()
    y_train = train_set[target_col].values.astype(float)

    X_val = val_set.drop(columns = [target_col]).values
    X_val_text = val_set[[text_column]].values.flatten().tolist()
    y_val = val_set[target_col].values.astype(float)

    X_test = test_set.drop(columns = [target_col]).values
    X_test_text = test_set[[text_column]].values.flatten().tolist()
    y_test = test_set[target_col].values.astype(float)

    return X_train, X_train_text, y_train, X_val, X_val_text, y_val, X_test, X_test_text, y_test

In [9]:
def create_tokens(X_train_text, X_val_text, X_test_text, text_column):
  transformer_model_name = "jplu/tf-xlm-roberta-base"
  tokenizer = XLMRobertaTokenizer.from_pretrained(transformer_model_name)
  max_length = 256

  #Tokenize training set
  train_input_ids = tokenizer(X_train_text, return_tensors="tf", max_length=max_length, padding="max_length", truncation=True)["input_ids"]
  train_attention_mask = tokenizer(X_train_text, return_tensors="tf", max_length=max_length, padding="max_length", truncation=True)["attention_mask"]

  #tokenize val set
  val_input_ids = tokenizer(X_val_text, return_tensors="tf", max_length=max_length, padding="max_length", truncation=True)["input_ids"]
  val_attention_mask = tokenizer(X_val_text, return_tensors="tf", max_length=max_length, padding="max_length", truncation=True)["attention_mask"]

  #tokenize test set
  test_input_ids = tokenizer(X_test_text, return_tensors="tf", max_length=max_length, padding="max_length", truncation=True)["input_ids"]
  test_attention_mask = tokenizer(X_test_text, return_tensors="tf", max_length=max_length, padding="max_length", truncation=True)["attention_mask"]

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_column}_train_input_ids", "wb") as f:
    pickle.dump(train_input_ids, f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_column}_train_attention_mask", "wb") as f:
      pickle.dump(train_attention_mask, f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_column}_val_input_ids", "wb") as f:
      pickle.dump(val_input_ids, f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_column}_val_attention_mask", "wb") as f:
      pickle.dump(val_attention_mask, f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_column}_test_input_ids", "wb") as f:
      pickle.dump(test_input_ids, f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_column}_test_attention_mask", "wb") as f:
      pickle.dump(test_attention_mask, f)

In [10]:
#STATISTICS ON THE TEXT
def text_statistics(df, upper_bound, text_column):
  values = np.array([len(text.split(" ")) for text in df[text_column].values])
  df_text_stats = pd.DataFrame(values)
  value = np.sort(values)[int(upper_bound * len(values))-1: int(upper_bound * len(values))]

  return df_text_stats.describe(), value

In [11]:
df.head()

,ISO_COUNTRY_CODE,CAE_TYPE,MAIN_ACTIVITY,B_ON_BEHALF,B_AWARDED_BY_CENTRAL_BODY,TYPE_OF_CONTRACT,TAL_LOCATION_NUTS,B_FRA_AGREEMENT,CPV,B_EU_FUNDS,TOP_TYPE,B_ACCELERATED,CRIT_CODE,CRIT_PRICE_WEIGHT,NUMBER_OFFERS,AWARD_EST_VALUE_EURO,AWARD_VALUE_EURO_FIN_1,DT_AWARD,LG_ORIG,short description,title,ac criterion,combined text,short_descr_criterion
0,BG,3,General public\services,N,N,W,BG314,N,31,N,OPE,N,M,50.0,3.0,2671496.57,2657688.93,2019-12-02,BG,Изготвяне на работен инвестиционен проект.,"Инженеринг — проектиране, упражняване на автор...",Срок за изпълнение Гаранционен срок,Изготвяне на работен инвестиционен проект. Сро...,Изготвяне на работен инвестиционен проект. Сро...
1,BG,1,Other,N,N,W,BG422,N,45,N,OPE,N,M,40.0,3.0,3067798.34,3058417.95,2019-12-09,BG,Обхватът на обществената поръчка включва следн...,"Инженеринг (проектиране, строителство и осъщес...",Оценка на техническото предложение Подпоказате...,Обхватът на обществената поръчка включва следн...,Обхватът на обществената поръчка включва следн...
2,GR,8,Health,N,N,U,EL434,N,38,N,OPE,N,M,100.0,1.0,242134.07,237356.48,2019-09-02,EL,«Συσκευές ανάλυσης για ανοσολογικές εξετάσεις ...,«Συσκευές ανάλυσης για ανοσολογικές εξετάσεις ...,Συμφωνία προσφοράς με τις τεχνικές προδιαγραφέ...,«Συσκευές ανάλυσης για ανοσολογικές εξετάσεις ...,«Συσκευές ανάλυσης για ανοσολογικές εξετάσεις ...
3,PL,3,General public\services,N,N,U,PL623,N,48,Y,OPE,N,M,60.0,3.0,389196.49,442493.81,2019-12-23,PL,Celem zamówienia realizowanego w ramach zadani...,"„Dostawa sprzętu komputerowego”, realizowanego...",Gwarancja,Celem zamówienia realizowanego w ramach zadani...,Celem zamówienia realizowanego w ramach zadani...
4,PL,8,Health,N,N,U,PL,N,33,N,OPE,N,L,100.0,1.0,67499.67,70968.59,2019-12-16,PL,1. Przedmiotem zamówienia jest dzierżawa 2 ana...,Dzierżawa 2 analizatorów immunologicznych wraz...,,1. Przedmiotem zamówienia jest dzierżawa 2 ana...,1. Przedmiotem zamówienia jest dzierżawa 2 ana...


In [12]:
df = df.loc[df["LG_ORIG"] == "ES"]
len(df)

6652

In [13]:
text_cols = ["short description", "short_descr_criterion", "combined text"]

In [14]:
for text_col in text_cols:
  print(text_col, ":", "\n", text_statistics(df, upper_bound = 0.995, text_column = text_col), "\n\n")

short description : 
 (                 0
count  6652.000000
mean     21.275857
std      15.328630
min       1.000000
25%      12.000000
50%      18.000000
75%      26.000000
max     171.000000, array([93])) 


short_descr_criterion : 
 (                 0
count  6652.000000
mean     41.338094
std      34.693283
min       1.000000
25%      20.000000
50%      32.000000
75%      50.000000
max     546.000000, array([217])) 


combined text : 
 (                 0
count  6652.000000
mean     58.273001
std      39.035065
min       3.000000
25%      33.000000
50%      51.000000
75%      72.000000
max     589.000000, array([242])) 




In [15]:
#X_train, X_train_text, y_train, X_val, X_val_text, y_val, X_test, X_test_text, y_test = encode_scale_split(df, 0.6, 0.2, text_column = "combined text");
#create_tokens(X_train_text, X_val_text, X_test_text, text_column="spanish subset")

<ipython-input-7-9d22b1199d63>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,numeric_cols] = np.array(scaled_values)
<ipython-input-7-9d22b1199d63>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,numeric_cols] = np.array(scaled_values)
<ipython-input-7-9d22b1199d63>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/512 [00:00<?, ?B/s]

In [15]:
def load_tokenized_text(text_col):
  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_col}_train_input_ids", "rb") as f:
    train_input_ids = pickle.load(f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_col}_train_attention_mask", "rb") as f:
    train_attention_mask = pickle.load(f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_col}_val_input_ids", "rb") as f:
    val_input_ids = pickle.load(f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_col}_val_attention_mask", "rb") as f:
    val_attention_mask = pickle.load(f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_col}_test_input_ids", "rb") as f:
    test_input_ids = pickle.load(f)

  with open(f"/content/drive/MyDrive/Thesis/1.data/{text_col}_test_attention_mask", "rb") as f:
    test_attention_mask = pickle.load(f)

  return train_input_ids, train_attention_mask, val_input_ids, val_attention_mask, test_input_ids, test_attention_mask

In [16]:
def RAE(y_true, y_pred):
    mean_true = sum(y_true) / len(y_true)
    delta_pred = sum(abs(true_value - pred_value) for true_value, pred_value in zip(y_true, y_pred))
    delta_mean = sum(abs(true_value - mean_true) for true_value in y_true)
    rae = float((delta_pred / delta_mean).item())
    return rae

In [17]:
def RMSE(y_true, y_pred):
    mean_true = sum(y_true) / len(y_true)
    delta_pred = sum((true_value - pred_value)**2 for true_value, pred_value in zip(y_true, y_pred))
    delta_mean = sum(abs(true_value - mean_true)**2 for true_value in y_true)
    rmse = float((delta_pred / delta_mean).item())
    return rmse

In [22]:
def create_train_model(model_name, batch_size, epochs, train_input_ids, train_attention_mask, y_train, val_input_ids, val_attention_mask, y_val, test_input_ids, test_attention_mask, y_test):

  results_total = {}
  #define path for saving model
  checkpoint_path = f"/content/drive/MyDrive/Thesis/2.models/spanish_checkpoint.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)
  n_batches = len(y_train) / batch_size
  n_batches = math.ceil(n_batches)

  # Create a callback that saves the model's weights
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                                 monitor="mae",
                                                                 save_weights_only=False,
                                                                 save_best_only=True,
                                                                 verbose=0,
                                                                 save_freq="epoch")

  #train the model directly on the classification task
  transformer_model_name = "jplu/tf-xlm-roberta-base"
  language_model = TFAutoModelForSequenceClassification.from_pretrained(transformer_model_name, num_labels=1)
  language_model.layers[0].trainable = False

  num_train_steps = len(y_train) / batch_size * epochs
  lr_scheduler = PolynomialDecay(
      initial_learning_rate=0.0005, end_learning_rate=0.000001, decay_steps=num_train_steps
  )

  optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
  metrics = ["mae", "mse", "R2Score"]

  #compile the model
  language_model.compile(optimizer="Adam", metrics = metrics, loss=keras.losses.MeanSquaredError())
  print(language_model.summary())

  #fit the model
  history = language_model.fit(x = [train_input_ids, train_attention_mask], y=y_train,
                                          validation_data=([val_input_ids, val_attention_mask], y_val),
                                          epochs=epochs,
                                          batch_size = batch_size)

  y_pred = language_model.predict([test_input_ids, test_attention_mask])
  mae_test = mean_absolute_error(y_test, y_pred[0])
  mse_test = mean_squared_error(y_test, y_pred[0])
  rae_test = RAE(y_test, y_pred[0])
  rmse_test = RMSE(y_test, y_pred[0])

  r2_metric = tf.keras.metrics.R2Score()
  r2_metric.update_state(y_test.reshape(-1, 1), y_pred[0])
  r2_result = r2_metric.result()
  r2_result = r2_result.numpy()

  results_total["language_model"] = {"history": history,
                                     "mae_test": mae_test,
                                     "mse_test": mse_test,
                                     "r2_results": r2_result,
                                     "rae_test": rae_test,
                                     "rmse_test": rmse_test}

  return results_total

In [23]:
results_total = {}

#LOAD TOKENIZED DATA
train_input_ids, train_attention_mask, val_input_ids, val_attention_mask, test_input_ids, test_attention_mask = load_tokenized_text("spanish subset")
X_train, X_train_text, y_train, X_val, X_val_text, y_val, X_test, X_test_text, y_test = encode_scale_split(df, 0.6, 0.2, text_column = "combined text");

#TRAIN MODEL ON DATA
results_total[text_col] = create_train_model(model_name = "spanish",
                                train_input_ids = train_input_ids,
                                train_attention_mask = train_attention_mask,
                                y_train = y_train,
                                val_input_ids = val_input_ids,
                                val_attention_mask = val_attention_mask,
                                y_val = y_val,
                                test_input_ids = test_input_ids,
                                test_attention_mask = test_attention_mask,
                                y_test = y_test,
                                batch_size = 32,
                                epochs = 5)

with open(f"/content/drive/MyDrive/Thesis/3.results/results_total_language_model_spanish", "wb") as f:
  pickle.dump(results_total, f)

<ipython-input-7-9d22b1199d63>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,numeric_cols] = np.array(scaled_values)
<ipython-input-7-9d22b1199d63>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,numeric_cols] = np.array(scaled_values)
<ipython-input-7-9d22b1199d63>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

tf_model.h5:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

Some layers of TFXLMRobertaForSequenceClassification were not initialized from the model checkpoint at jplu/tf-xlm-roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tfxlm_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFXLMRobertaMainL  multiple                  277453056 
 ayer)                                                           
                                                                 
 classifier (TFXLMRobertaCl  multiple                  591361    
 assificationHead)                                               
                                                                 
Total params: 278044417 (1.04 GB)
Trainable params: 591361 (2.26 MB)
Non-trainable params: 277453056 (1.03 GB)
_________________________________________________________________
None
Epoch 1/5
125/125 [==============================] - 52s 287ms/step - loss: 1210860568576.0000 - mae: 611565.6250 - mse: 1210860568576.0000 - r2_score: -0.4469 - val_loss: 839785971712.0000 - val_mae: 529363.0000 - val_mse: 839785971712.00

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [ ]:
directory = "/content/drive/MyDrive/Thesis/2.models/"
model_paths = [f"/content/drive/MyDrive/Thesis/2.models/{model}" for model in os.listdir(directory)]
model_paths

[]

In [24]:
results_total

{'combined text': {'language_model': {'history': <keras.src.callbacks.History at 0x7a72b1a078e0>,
   'mae_test': 559718.7269855061,
   'mse_test': 971543790809.8317,
   'r2_results': -0.47592998,
   'rae_test': 1.0983939170837402,
   'rmse_test': 1.47593092918396}}}

In [ ]:
def get_test_results(y_test, X_test):
    #LOAD BEST MODELS AND MAKE PREDICTIONS
  directory = "/content/drive/MyDrive/Thesis/2.models/"
  model_paths = [f"/content/drive/MyDrive/Thesis/2.models/{model}" for model in os.listdir(directory)]

  model_test_results = {}

  for path in tqdm(model_paths):
      loaded_model = AutoModelForSequenceClassification.from_pretrained(path)

      #determine metrics
      y_pred = model.predict(X_test, verbose = 0)
      mae_test = mean_absolute_error(y_test, y_pred)
      mse_test = mean_squared_error(y_test, y_pred)
      rae_test = RAE(y_test, y_pred)
      rmse_test = RMSE(y_test, y_pred)

      r2_metric = tf.keras.metrics.R2Score()
      r2_metric.update_state(y_test.reshape(-1, 1), y_pred)
      r2_result = r2_metric.result()
      r2_result = r2_result.numpy()

      model_test_results[path.split("/")[2]] = {"mae_test": mae_test,
                        "mse_test": mse_test,
                        "r2_results": r2_result,
                        "rae_test": rae_test,
                        "rmse_test": rmse_test}

  with open("/content/drive/MyDrive/Thesis/3.results/model_test_results_all", "wb") as f:
      pickle.dump(model_test_results, f)

  return model_results

In [ ]:
model_results = get_test_results(y_test, X_test)

0it [00:00, ?it/s]


NameError: name 'model_results' is not defined

In [ ]:
model_results